In [2]:
#------------------------------- PROBLEM 1A ----------------------------------#
from tracemalloc import start
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import time
import matplotlib.pyplot as plt

# Check for GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the second GPU
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

# Data loading and transformations
data_path = '../data-unversioned/p1ch7/'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

# CNN Neural Network using nn.Sequential
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(32 * 16 * 16, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
).to(device)


# Training
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

training_losses = []
validation_losses = []

overall_start_time = time.time()

for epoch in range(n_epochs):
    start_time = time.time()
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    total_train_samples = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs, dim=1)
        total_train_correct += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = total_train_correct / total_train_samples
    end_time = time.time()
    epoch_time = end_time - start_time

    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            total_val_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            total_val_correct += (predicted == labels).sum().item()
            total_val_samples += labels.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = total_val_correct / total_val_samples

    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Time: {epoch_time:.2f} seconds")

overall_end_time = time.time()
total_training_time = overall_end_time - overall_start_time

print(f"Total training time: {total_training_time:.2f} seconds")


Using GPU: NVIDIA GeForce MX550
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Training Loss: 1.7201, Training Accuracy: 0.3934, Validation Loss: 1.6453, Validation Accuracy: 0.3920, Time: 8.73 seconds
Epoch: 2, Training Loss: 1.3727, Training Accuracy: 0.5135, Validation Loss: 1.3601, Validation Accuracy: 0.5118, Time: 8.27 seconds
Epoch: 3, Training Loss: 1.2271, Training Accuracy: 0.5693, Validation Loss: 1.2869, Validation Accuracy: 0.5417, Time: 8.29 seconds
Epoch: 4, Training Loss: 1.1374, Training Accuracy: 0.5995, Validation Loss: 1.2552, Validation Accuracy: 0.5580, Time: 8.48 seconds
Epoch: 5, Training Loss: 1.0630, Training Accuracy: 0.6269, Validation Loss: 1.1959, Validation Accuracy: 0.5682, Time: 8.57 seconds
Epoch: 6, Training Loss: 0.9935, Training Accuracy: 0.6548, Validation Loss: 1.1481, Validation Accuracy: 0.5852, Time: 8.58 seconds
Epoch: 7, Training Loss: 0.9296, Training Accuracy: 0.6779, Validation Loss: 1.1063, Validatio

In [ ]:

plt.figure(figsize=(12, 5))
plt.plot(training_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()